## 1. url, likes crawler

In [1]:
import time
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from inscrawler import secret


username = 'streetstylestars'


def find_one(driver, css_selector, waittime=0, elem=None):
    if waittime:
        WebDriverWait(driver, waittime).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css_selector))
        )
    return driver.find_element(By.CSS_SELECTOR, css_selector)


# Settings
service_args = ["--ignore-ssl-errors=true"]
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--no-sandbox")

# Open browser
driver = webdriver.Chrome(
    executable_path="inscrawler/bin/chromedriver",
    service_args=service_args,
    chrome_options=chrome_options,
)
driver.implicitly_wait(5)

# log in instagram
url = 'https://www.instagram.com/accounts/login'
driver.get(url)

u_input = find_one(driver, 'input[name="username"]', 0.5)
u_input.send_keys(secret.username)
p_input = find_one(driver, 'input[name="password"]', 0.5)
p_input.send_keys(secret.password)

login_btn = find_one(driver, ".L3NKy", 0.5)
login_btn.click()
time.sleep(3)

# Get target page
url = 'https://www.instagram.com/' + username
driver.get(url)
driver.implicitly_wait(5)

# Render enough posts
for i in range(15):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(1.5)

## Restart here when TimeoutException
## (before run, manually click 'X' - exit button at the browser)

In [2]:
# click first post
driver.execute_script("window.scrollTo(0, 0)")
ele_post = find_one(driver, ".v1Nh3 a", 0.5)
ele_post.click()
time.sleep(1.5)

# crawl url, likes of 180 posts
res_list = []
for i in range(180):
    tmp_dict = {}
    
    tmp_url = None
    likes = 0
    try:
        ele_img = find_one(driver, '._97aPb img', 10)
        tmp_url = ele_img.get_attribute("src")
        
        ele_likes = find_one(driver, '.Nm9Fw > * > span', 10)
        likes = ele_likes.text
        likes = int(likes.replace(",", "").replace(".", "")) if likes is not None else 0
    except:
        pass
    tmp_dict['img_url'] = tmp_url
    tmp_dict['likes'] = likes
    
    res_list.append(tmp_dict)
    
    # move to next post
    next_btn = find_one(driver, ".EfHg9 ._65Bje.coreSpriteRightPaginationArrow", 0.5)
    next_btn.click()
    time.sleep(0.5)
    
    # process check
    if (i+1)%50==0:
        print(f'>>> {i+1}th src is crawled successfully')

print('\n===== crawling successfully finished =====')

>>> 50th src is crawled successfully
>>> 100th src is crawled successfully
>>> 150th src is crawled successfully

===== crawling successfully finished =====


In [3]:
print(f'>>> # of items    : {len(res_list)}')

out = json.dumps(res_list, ensure_ascii=False)
with open('./output/output_'+username, "w", encoding="utf8") as f:
    f.write(out)

>>> # of items    : 180


## =====================================
## 2. picture crawler

In [4]:
import os
from urllib.request import urlretrieve


def retrieve_pictures(username, likes_threshold=0, no_pictures=150):
    # Open the output binary file of crawler.py
    with open('./output/output_'+username, encoding='utf-8') as fp:
        data = fp.read()
        pass
    data_list = json.loads(data)
    print("json file successfully loaded")

    # Make a new directory to save pictures
    dir_path = './output/pictures_'+username
    if not os.path.exists(dir_path):
            os.mkdir(dir_path)
            print("Directory " , dir_path ,  " Created")
    else:    
            print("Directory " , dir_path ,  " already exists")

    # retrieve & save pictures from instagram
    # (only picutres with likes more than likes numbers)
    success_num = 0
    for idx, item in enumerate(data_list):
        idx += 1

        tmp_likes = item['likes']
        if tmp_likes < likes_threshold:
            continue

        if item['img_url'] is not None:
            tmp_url = item['img_url']

            title_idx = '00'+str(idx)
            title_idx = title_idx[-3:]
            urlretrieve(tmp_url, dir_path+f'/{username}_{title_idx}_{tmp_likes}Likes.png')
            
            success_num += 1

        if (idx+1) % 50 == 0:
            print(f'>>> picture {idx+1} has been successfully downloaded.')
        
        if success_num == no_pictures:
            break
        pass
    print(f'download successfully completed')


retrieve_pictures(username)

json file successfully loaded
Directory  ./output/pictures_fashionandstyle.official  Created
>>> picture 50 has been successfully downloaded.
>>> picture 100 has been successfully downloaded.
>>> picture 150 has been successfully downloaded.
download successfully completed
